In [1]:
import datetime

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession

In [2]:
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
# import pandas as pd

# # import data
# data = pd.read_csv("/home/bluepi/Downloads/Update/product_info/main table/product_info.csv")

# # convert day column type to datetime
# data['day'] = pd.to_datetime( data['day'], infer_datetime_format=True, yearfirst=True)

# # create new column
# data['date_timestamp'] = pd.to_datetime(data.day.astype(str) + ' ' + data.time)

# # Drop old columns
# data.drop(['day','time'],inplace=True,axis=1)

# # Write to csv
# data.to_csv("/home/bluepi/Downloads/Update/Updated Product/Latest Product/main table.csv", index=False)

In [7]:
mainTable = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main table.csv")

In [ ]:
mainTable.printSchema()

In [8]:
from pyspark.sql.functions import year, month, dayofyear, hour, minute, second

mainTable.select([year(mainTable['date_timestamp']).alias("Year"),
                  month(mainTable['date_timestamp']).alias("Month"),
                  dayofyear(mainTable['date_timestamp']).alias("DayOfYear"),
                  hour(mainTable['date_timestamp']).alias("Hour"),
                  minute(mainTable['date_timestamp']).alias("Minute"),
                  second(mainTable['date_timestamp']).alias("Second")]).show(3)

+----+-----+---------+----+------+------+
|Year|Month|DayOfYear|Hour|Minute|Second|
+----+-----+---------+----+------+------+
|2020|    5|      123|   8|     6|    42|
|2020|    2|       33|   8|     2|    22|
|2020|    2|       59|   7|    58|     8|
+----+-----+---------+----+------+------+
only showing top 3 rows



In [9]:
mainTable.select('date_timestamp').orderBy(mainTable.date_timestamp.asc()).show(6)

+-------------------+
|     date_timestamp|
+-------------------+
|2020-01-02 01:25:16|
|2020-01-02 05:05:59|
|2020-01-02 05:54:01|
|2020-01-02 09:08:05|
|2020-01-02 20:11:17|
|2020-02-02 01:16:34|
+-------------------+
only showing top 6 rows



In [18]:
# Address to the product_info folder
address = "/home/bluepi/Downloads/Update/product_info/"
previous_day = (datetime.datetime.today() - datetime.timedelta(days=2)).strftime('%d-%m-%Y')

# Address to the Previous Day folder
new_address = address + previous_day

# Read the Previous Day folder
per_day_data = spark.read.format('csv') \
          .options( header=True, inferschema=True ) \
          .load(new_address)

In [19]:
# per_day_data.show(3)
per_day_data.orderBy(per_day_data.Date_timestamp.asc()).show(60)
# per_day_data.printSchema()

+----+---------+-----+-------------------+-----------+
|p_id|   p_name|price|     Date_timestamp|record_type|
+----+---------+-----+-------------------+-----------+
| 119|     Rank|  531|2020-03-24 01:29:25|          U|
| 197|   Sub-Ex|   76|2020-03-24 01:30:59|          U|
| 135|  Cardify|  974|2020-03-24 01:34:15|          D|
| 137|   Tresom| 1459|2020-03-24 01:36:14|          U|
| 141|Lotstring|  273|2020-03-24 01:45:29|          D|
| 224| Wrapsafe|  935|2020-03-24 02:00:11|          I|
| 206|    Alpha| 1182|2020-03-24 02:00:13|          I|
| 193| Treeflex|  297|2020-03-24 02:00:24|          D|
|  91|  Konklab|  465|2020-03-24 02:03:21|          U|
| 216|   Lotlux|  613|2020-03-24 02:15:15|          I|
| 245| Home Ing| 1127|2020-03-24 07:31:48|          I|
|  94|    Opela|  994|2020-03-24 07:50:10|          D|
| 242| Greenlam| 1942|2020-03-24 07:55:26|          I|
| 168|  Bitchip| 1160|2020-03-24 08:03:16|          U|
| 200|  Regrant| 1369|2020-03-24 08:04:16|          U|
| 136|Hold

In [20]:
# # Create the schema
# from pyspark.sql.types import *

# schema = StructType([StructField("p_id", IntegerType(), True),
#                      StructField("p_name", StringType(), True),
#                      StructField("price", IntegerType(), True),
#                      StructField("Date_timestamp", TimestampType() , True),
#                      StructField("record_type", StringType(), True)
#                      ])
# # Create latest proct table
# # Initially it is empty no updates
# Latest_Product_Table = spark.createDataFrame([], schema)

In [26]:
# Directly append new Inserted products
from pyspark.sql.functions import *
per_day_data_1 = per_day_data.filter(
    per_day_data.record_type == 'U').select(['p_id', 'date_timestamp'])

ppd_1 = per_day_data_1.alias('ppd_1')
ppd_2 = per_day_data_1.alias('ppd_2')

per_day_data_2 = ppd_1.join(
    ppd_2, ppd_1.p_id == ppd_2.p_id, 'inner').where(ppd_1.date_timestamp > ppd_2.date_timestamp)

per_day_data_2.show()
# per_day_data_2 = per_day_data_1.select(
#     ['p_id', 'p_name', 'price', 'Date_timestamp'])
# mainTable_1 = mainTable.union(per_day_data_2)

# # mainTable_1.count()

+----+--------------+----+--------------+
|p_id|date_timestamp|p_id|date_timestamp|
+----+--------------+----+--------------+
+----+--------------+----+--------------+



In [30]:
ppd_2.select("hour("date_add_timestamp").select()

SyntaxError: invalid syntax (<ipython-input-30-6ccdc94c2f19>, line 1)

In [12]:
# Drop deleted products

# Registers this DataFrame as a temporary table using the given name
mainTable_1.registerTempTable("table_main")
per_day_data.registerTempTable("table_day")

mainTable_2 = spark.sql(
    "select * from table_main where p_id not in ( select p_id from table_day where record_type = 'D' )")

# mainTable_2.count()

NameError: name 'mainTable_1' is not defined

In [ ]:
# update products
from pyspark.sql.functions import *
mainTable_2.select( countDistinct(mainTable_2.p_id) ).show()

In [ ]:
mainTable_2.select('p_id').groupBy("p_id").count().filter( "count > 1" ).show()